# **Fourier Series and spectra**

With *Fourier series*, we can approximate a periodic function $x(t)$ as a weighted summation of sinusoidal basis functions. 

$$ x(t) \approx A_0 + \sum_{k=1}^N \left(A_k\cos(\omega_k t + \varphi_k) \right)$$

This decomposition allows us to analyze a signal in terms of its frequency components, which is fundamental in many applications, such as spectral filtering, image compression, and solving partial differential equations.

Note that, in addition to the Fourier series, there are other types of series expansions. In a *Taylor series*, the basis functions are polynomials of increasing order. In a *wavelet series*, the basis functions are wavelets—special functions that are localized in both time and frequency. What this means in practice, we will explore later in the course.

For now, we will focus entirely on the Fourier series. Let’s get started!

## **Exercises**
* [Exercise 1 – Periodic signals](#exercise1)  
* [Exercise 2a – Compute Fourier series](#exercise2a)
* [Exercise 2b – Look up Fourier coefficients](#exercise2b)  
* [Exercise 3 – Spectrum](#exercise3)  
* [Exercise 4 – Inverse Fourier / signal reconstruction](#exercise4)  
* [Exercise 5 – Fourier spectrum using FFT](#exercise5)  
* [Exercise 6 – Fourier and aliasing](#exercise6)  

---

## **Preparations**

Let's start with the usual preparatory steps...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from scipy.fft import fft, ifft, fftfreq, fftshift

# Jupyter / IPython configuration:
# Automatically reload modules when modified
%load_ext autoreload
%autoreload 2

# Enable vectorized output (for nicer plots)
%config InlineBackend.figure_formats = ["svg"]

import sys
sys.path.insert(0, "../")
import isp

---

<a id='exercise1'></a>

## **&#9734;  Exercise 1 – Periodic signals**

In the lectures, we have explored various periodic functions $x(t)$, such as the sawtooth and pulse functions.

The [`scipy.signal`](https://docs.scipy.org/doc/scipy/reference/signal.html#waveforms) module provides already built-in support for generating several types of waveforms. 
 

In [ ]:
from scipy import signal

# Illustration of different waveforms offered by numpy and scipy.signal:
duration = 1    # seconds
f_signal = 5    # Hz
f_sample = 1000 # Hz

t = np.arange(0, duration, 1/f_sample)

# Sine wave
x_sine = np.sin(2 * np.pi * f_signal * t)
x_rect = signal.square(2 * np.pi * f_signal * t, duty=0.3)
x_sawtooth = signal.sawtooth(2 * np.pi * f_signal * t)
x_triangle = signal.sawtooth(2 * np.pi * f_signal * t, width=0.5)

# Decorate the plot
def decorate(ax):
    ax.set_ylabel("x(t)")
    # Add legend outside of the plot
    ax.legend(loc="upper left", bbox_to_anchor=(1, 1.05))
    # Add a zero line
    ax.plot([0, duration], [0, 0], "k--", lw=0.5, alpha=0.5)

# Plot
fig, axes = plt.subplots(4, 1, figsize=(6, 6), sharex=True)
axes[0].plot(t, x_sine, label="Sine")
axes[1].plot(t, x_rect, label="Rectangular")
axes[2].plot(t, x_sawtooth, label="Sawtooth")
axes[3].plot(t, x_triangle, label="Triangle")
for ax in axes:
    decorate(ax)
# Only for last ax:
ax.set_xlabel("Time [s]");


Your task is to construct these signals using only NumPy or basic Python operations.

### **Instructions**
* Complete the following functions.
* Visualize the results to verify correctness.
* Hints:
  * The modulo operation `t%T` computes the remainder when dividing `t` by `T`. This can also be used for an array of values `t`.
  * The modulo operation `t%T` computes the remainder of `t` divided by `T`. When applied to an array `t`, it operates element-wise.
  * [`np.where(cond, x, y)`](https://numpy.org/doc/stable/reference/generated/numpy.where.html) assigns a value `x` if condition `cond` is True, and `y` otherwise.

In [ ]:
######################
###    EXCERISE    ###
######################

def rectangular_wave(t, f, duty=0.5):
    """Generate a rectangular wave.

    Parameters
    ----------
    t : array_like      Time vector
    f : float           Frequency of the wave
    duty : float        Duty cycle of the wave
    """
    ...


def sawtooth_wave(t, f):
    """Generate a sawtooth wave.

    Parameters
    ----------
    t : array_like      Time vector
    f : float           Frequency of the wave
    """
    ...


def triangle_wave(t, f):
    """Generate a triangle wave.

    Parameters
    ----------
    t : array_like      Time vector
    f : float           Frequency of the wave
    """
    ...


# Plot
fig, axes = plt.subplots(3, 1, figsize=(6, 6), sharex=True)
axes[0].plot(t, rectangular_wave(t, f_signal, duty=0.3), label="Rectangular")
axes[1].plot(t, sawtooth_wave(t, f_signal), label="Sawtooth")
axes[2].plot(t, triangle_wave(t, f_signal), label="Triangle")
for ax in axes:
    decorate(ax)
# Only for last ax:
ax.set_xlabel("Time [s]");

---

<a id='exercise2a'></a>

## **&#9734;&#9734;  Exercise 2a – Compute Fourier series**

In the lecture, we introduced the procedure for calculating Fourier coefficients. The following formulas are fundamental

$$\begin{align*} 
a_0&= \frac{1}{T}\int_{-T/2}^{T/2} x(t)\,dt\\
a_k &= \frac{2}{T}\int_{-T/2}^{T/2} x(t) \cos \left( 2\pi \tfrac{k}{T} t \right) \,dt\qquad\text{for } k\geq 1\qquad\\
b_k &= \frac{2}{T}\int_{-T/2}^{T/2} x(t) \sin \left( 2\pi \tfrac{k}{T} t \right) dt\qquad\text{for } k\geq 1
\end{align*}
$$

These integrals allow us to compute the Fourier series coefficients (in sine-cosine form), which characterize a function in terms of its frequency components. 

### **Example: Fourier Series of a square wave**
Let's apply these formulas to a square wave (a rectangular wave with duty cycle = 0.5). Since the Fourier transform only considers one period at a time, we define:

$$
x(t) = \begin{cases} -1 & \text{if } &-\frac{T}{2} \leq t < 0 
\\ +1 & \text{if }  &0 \leq t < \frac{T}{2} \end{cases} $$

It is important to note that the choice of period interval has no influence on the result. The integrals could also be evaluated using $0 \leq t \leq T$, for example. 


#### **Coefficient $a_0$ (DC component)**

$$
\begin{align*}
a_0 &= \frac{1}{T}\left(\int_{-T/2}^0 (-1)\,dt + \int^{T/2}_0 (+1)\,dt\right)\\
&= \frac{1}{T}\left(-t\,\bigg\rvert_{-T/2}^0 + t\,\bigg\rvert_{0}^{T/2}\right)\\
&= 0
\end{align*}
$$

The term $a_0$​ represents the DC component (the average value of the signal). Since the square wave is symmetric around zero, the average value is indeed zero. 


#### **Coefficient $a_k$ (cosine coefficients)**

$$
\begin{align*}
a_k &= \frac{2}{T}\left(\int_{-T/2}^0 (-1)\cos\left(\tfrac{2\pi k}{T}t\right) \,dt + \int^{T/2}_0 (+1)\cos\left(\tfrac{2\pi k}{T}t\right)\,dt\right)\\
&= -\frac{2}{T}\frac{T}{2\pi k}\sin\left(\frac{2\pi k}{T}t\right)\,\bigg\rvert_{-T/2}^0 + 
\frac{2}{T}\frac{T}{2\pi k}\sin\left(\frac{2\pi k}{T}t\right)\,\bigg\rvert_{-T/2}^0\\
 &= 0
\end{align*}
$$

All coefficients $a_k$  are zero because the square wave is an odd function, while cosine functions are even. The $a_k$ contribute only to the *even* components in the signal, which are not present in the square wave signal. 


#### **Coefficient $b_k$ (sine coefficients)**


$$
\begin{align*}
b_k &= \frac{2}{T}\left(\int_{-T/2}^0 (-1)\sin\left(\tfrac{2\pi k}{T}t\right) \,dt + \int^{T/2}_0 (+1)\sin\left(\tfrac{2\pi k}{T}t\right)\,dt\right) \\
&= \frac{2}{T}\frac{T}{2\pi k}\cos\left(\frac{2\pi k}{T}t\right)\,\bigg\rvert_{-T/2}^0 
- \frac{2}{T}\frac{T}{2\pi k}\cos\left(\frac{2\pi k}{T}t\right)\,\bigg\rvert_0^{T/2} \\[1em]
&= \frac{1}{\pi k}(\cos(0)- \cos(-\pi k)) - \frac{1}{\pi k}(\cos(\pi k)- \cos(0))\\[0.75em]
&= \frac{1}{\pi k}(1 - \cos(+\pi k)) - \frac{1}{\pi k}(\cos(\pi k)- 1)\\[0.75em]
&= \begin{cases} 0, & k\text{ even}\\
\frac{4}{\pi k}, & k\text{ odd}
\end{cases}
\end{align*}
$$

In the last lines, we used the following facts:
* $\cos(-t)=\cos(t)$
* $\cos({\pi k})=(-1)^k$ 

Thus, the square wave consists only of sine terms with odd harmonic frequencies.

<br>

### **Instructions**
* Try to follow the derivation and understand why certain coefficients vanish.
* For the brave ones only: Repeat the process for a sawtooth wave.



```lang-none
######################
###    EXCERISE    ###
######################
````

Solve this task with paper and pencil (or a tablet).
<br><br><br><br><br>



---

<a id='exercise2b'></a>

## **&#9734;  Exercise 2b – Look up Fourier coefficients**

Alternatively, instead of deriving the Fourier coefficients manually, we can look up the coefficients for special functions $x(t)$ in a formulary.

### **Instructions**
* Search the web for the Fourier series coefficients of 
  * the *rectangular wave*, 
  * the *sawtooth wave* and 
  * the *triangular wave*.
* Use the *sine-cosine form* of the Fourier series!
* Make sure to really match the function $x(t)$ that we introduced above!


```lang-none
######################
###    EXCERISE    ###
######################
````

Type here your solution.


***Formulas for rectangular wave***

$a_0 = ... $  
$a_k = ... $  
$b_k = ... $  

***Formulas for sawtooth wave***

$a_0 = ... $  
$a_k = ... $  
$b_k = ... $  

***Formulas for traingular wave***

$a_0 = ... $  
$a_k = ... $  
$b_k = ... $  

<br>

---

<a id='exercise3'></a>

## **&#9734;  Exercise 3 – Spectrum**

The **amplitude spectrum** of a signal describes how strongly the harmonics (integer multiples of the fundamental frequency $f=1/T$​) contribute to the signal. It provides insight into the amplitude distribution of the signal in the frequency domain.

The phase spectrum, on the other hand, indicates the relative phase shifts of these harmonics, determining their position in the time domain.

In the previous exercise, we determined the Fourier coefficients of periodic signals (rectangular, sawtooth, triangular) using the sine-cosine form. To compute the magnitude (amplitude) and phase of the spectrum, we use the following conversion formulas:


$$
\begin{align*}
A_k &= \sqrt{a_k^2 + b_k^2} \\[0.5em]
\varphi_k&=\arctan{\left(\frac{b_k}{a_k}\right)}
\end{align*}
$$


### **Instructions**
* First answer this conceptual question: What are the angular frequencies $\omega_k$ associated with the Fourier coefficients?
* Given the expressions for $a_k, b_k$:
  * Plot the magnitude spectrum $A_k$
  * Plot the phase spectrum $\varphi_k$
  * Use [`plt.stem()`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.stem.html) to visualize the discrete spectrum

Use a square wave with a period of one second as the example signal.

In [ ]:
######################
###    EXERCISE    ###
######################

# Functions that provide the Fourier series coefficients for the square wave.
def get_a0(): return 0
def get_ak(k): return np.zeros_like(k)
def get_bk(k): return np.where(k%2 != 0, 4 / (np.pi * k), 0) 

k = np.arange(1, 10)
a0 = get_a0()
ak = get_ak(k)
bk = get_bk(k)
omega = ...

Ak = ...
phase_k = ...

# Plot the magintude spectrum
...

# Plot the phase spectrum
...




<span style="color:gray">
Note the discrepancy between the phase computed with the Fast Fourier Transform (FFT) and the mathematically derived phase (±π/2) for a square wave. This arises from how the FFT processes discrete, finite-length signals. The mathematical Fourier series assumes an infinitely periodic function that is symmetrically centered around t=0. In contrast, the FFT operates on a finite segment of the signal, assuming it starts at t=0, which can introduce phase shifts.
<br>

**Take-home message**: While the FFT resembles the Fourier series, they are not the same. The Fourier series applies to continuous-time periodic signals, while the Discrete Fourier Transform (DFT) — which the FFT efficiently computes — applies to discrete-time, non-periodic signals. Although conceptually related, they have fundamental differences that impact the computed spectra.
</span>.


---

<a id='exercise4'></a>

## **&#9734;  Exercise 4 – Inverse Fourier / signal reconstruction**

A Fourier series can be used to approximate a periodic function $x(t)$ by summing a finite number of terms:

$$x(t) \approx a_0 + \sum_{k=1}^N a_k\cos\left(\tfrac{2\pi k}{T}t\right)$$

The more terms we include, the more accurately the approximation captures the original signal.

In this exercise, we will reconstruct the signal using a truncated Fourier series.

### **Instructions**
* Use the Fourier coefficients of the square wave (with period $T=1s$).
* Approximate $x(t)$ using the partial sum for $N=6$.  
  (This requires implementing the formula above.)
* Plot the original signal and compare it with its reconstructed version for $N=6$.
* Use a sampling frequency of $f_{recon} = 1000$ and a duration of 5 seconds for the reconstructed signal.

In [ ]:
######################
###    EXERCISE    ###
######################
N=5
duration = 5
f_recon = 1000
k = np.arange(1, N+1)
a0 = ...
ak = ... 
bk = ...

t = ...

x_recon = ...
x_orig = ...


# Plot
fig, ax = plt.subplots()
ax.plot(t, x_recon, label="Reconstructed")
ax.plot(t, x_orig, label="Original")
plt.show()

---

<a id='exercise5'></a>

## **&#9734;  Exercise 5 – Fourier spectrum using FFT**

A little preview: We can compute discrete Fourier transforms for (almost) any discrete signal using the Fast Fourier Transform (FFT) algorithm. We will explore this in more detail in upcoming lectures, but for now, we will already use it in the context of Fourier series.

### **Instructions**
- First: Skim through [the Scipy tutorial on FFTs](https://docs.scipy.org/doc/scipy/tutorial/fft.html). Do you understand the main differences to Fourier series?
- Create a composite signal with the following characteristics:
  - (At least) three harmonic components with frequencies (10, 100, 200)Hz
  - Amplitudes (2, 1.5, 3)
  - with a DC component of amplitude = 2
- Use a sampling frequency of $f_s=1$ kHz to construct the signal
- Compute the Fourier spectrum of this composite signal. Here is the procedure:
    1. Compute the `fft` by passing the signal: `X = fft(x)`  
      This creates the discrete Fourier transform $X[k] = \mathcal{F}(x(t)$)
    2. Compute the corresponding frequencies $\omega_k$ using `freqs = fftfreq(N, 1/fs)`.  
    We need to specify the number of samples $N$, and the sampling period $\Delta t = 1/f_s$.
    3. For practical reasons, the frequencies are sorted slightly awkwardly. Calling `fftshift()` on both the Fourier transform `X` and the frequencies `freqs`, fixes this.
- Visualize the Fourier spectrum
    - Plot the full (symmetric) spectrum, ensuring the frequency and amplitude axes are correctly labeled.
    - Separate amplitude and phase plots to analyze signal properties.
- Advanced task: 
    - Add random noise with an amplitude of 0.5 to the composite signal.
    - Compute and visualize the Fourier spectrum of the noisy signal.
    - Display only the positive frequencies (use rfftfreq() for this).
    - Compare and discuss the differences in the spectra before and after adding noise.


In [ ]:
######################
###    EXERCISE    ###
######################

# Define parameters
duration = 2  # Signal duration in seconds
fs = 1000     # Sampling frequency
t = np.arange(0, duration, 1/fs)  # Time vector

# Construct a composite signal as specified
x = ...

# Compute the Fourier transform
xf = ...
freqs = ...
...

# Plot the magnitude and phase spectrum
...

# Construct a noisy signal
x_noisy = x + np.random.normal(0, 0.5, len(x))

# Repeat: Compute the Fourier transform
xf_noisy = ...
freqs_noisy = ...

# Plot the magnitude and phase spectrum
...

---

<a id='exercise6'></a>

## **&#9734;&#9734; Exercise 6 – Fourier and aliasing**

This exercise demonstrates the effect of aliasing on the Fourier transform of a signal. Aliasing occurs when a signal is sampled at a rate lower than the Nyquist frequency (twice the highest frequency present in the signal), leading to distortions in the frequency domain.

### **Instructions**
- Construct a composite signal `x_noisy` similar to the previous example, including noise.
- Downsample the signal using a sampling frequency below the Nyquist limit (e.g., $fs=120\mathsf{Hz}$).
- Compute and compare the Fourier transform of both the original and downsampled signals.
- Analyze and visualize the impact of aliasing in the amplitude spectrum.
- Describe your observations


In [ ]:
######################
###    EXERCISE    ###
######################

fs_resampled = 120
t_resampled = np.arange(0, duration, 1/fs_resampled)
x_resampled = ...

# Compute the Fourier transform
xf_resampled = ...
freqs_resampled = ...

# Plot the magnitude and phase spectrum
...